# Backtesting Analysis
This project is to demonstrate how to utilize backtesting to assess stock performance, further increasing the likelihood and filter down to the best stock candidates. 

## What is Backtesting?
Backtesting with Simple Moving Averages (SMAs) in Python is a good step in evaluating the effectiveness of an investment strategy before risking real money in the financial markets. This approach use simulating trades using historical price data to assess how a strategy would have performed in the past. 

By applying these rules to historical price data, key performance metrics can be measured, such as returns, drawdowns, and risk-adjusted ratios. These results provide valuable insights into the strategy's historical profitability and risk profile, helping traders make informed decisions. To summarize, backtesting gives sort of a conclusion to investors where `"if a stock is performing well in the past, it is more likely to perform well in the future.`

This Backtesting Analysis can be done with the use of a library called `Backtesting.py` which simplifies the process by passing the stock listing of choice and its history, the number of cash to invest, and the method of strategy to be used as an argument.

In [126]:
# import libraries
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## Retrieving the stock symbol candidate historical prices (yfinance)
The Backtesting.py library requires a "bring your own data" approach which can be easily retrieved through Yahoo Finance with yfinance API. The historical prices are retrieved which will be used to be analyzed. 

To demonstrate, one of the potential candidates that successfully went through the filtration by its dividend and foundational health (class A or B) will be used for this backtesting. A 1 year period is simulated for this demonstration.

In [132]:
# Example OHLC daily data for Google Inc.
import backtesting
from backtesting.test import GOOG
backtesting.set_bokeh_output(notebook=False)

ticker = yf.Ticker("6139.KL")

In [133]:
hist = ticker.history(period="1y")

print(hist)

                               Open      High       Low     Close   Volume  \
Date                                                                         
2022-10-04 00:00:00+08:00  3.049451  3.097549  3.049451  3.097549   164800   
2022-10-05 00:00:00+08:00  3.126408  3.145648  3.116789  3.126408   704300   
2022-10-06 00:00:00+08:00  3.116789  3.155267  3.116789  3.155267   229800   
2022-10-07 00:00:00+08:00  3.136028  3.174507  3.126408  3.164887   427000   
2022-10-11 00:00:00+08:00  3.145648  3.145648  3.087930  3.126408   708500   
2022-10-12 00:00:00+08:00  3.097549  3.116789  3.078310  3.097549   644900   
2022-10-13 00:00:00+08:00  3.097549  3.126408  3.068690  3.078310   814800   
2022-10-14 00:00:00+08:00  3.078310  3.136028  3.078310  3.116789   580400   
2022-10-17 00:00:00+08:00  3.097549  3.174507  3.097549  3.136028   413000   
2022-10-18 00:00:00+08:00  3.126408  3.126408  3.107169  3.126408    92700   
2022-10-19 00:00:00+08:00  3.126408  3.203366  3.126408  3.18412

## Defining Backtesting Strategy with Simple Moving Average

This Python backtesting code implements a simple moving average (SMA) crossover trading strategy using the Backtesting library. In this strategy, two SMAs are defined: a shorter-term SMA (n1) and a longer-term SMA (n2). The key idea is to use these SMAs to identify potential buy and sell signals based on their crossovers.


- When the shorter-term SMA (sma1) crosses above the longer-term SMA (sma2), it triggers a buy signal. Any existing positions (long or short) are closed, and a long position (buy) is initiated.
- Conversely, when sma1 crosses below sma2, it triggers a sell signal. Again, existing positions are closed, and a short position (sell) is initiated.

This strategy aims to capture potential trends in the asset's price movements by leveraging SMA crossovers. Backtesting is then used to evaluate the strategy's historical performance based on these rules, providing insights into its profitability and effectiveness.

In [134]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [135]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

## Running the Backtest

In [136]:
# from backtesting import Backtest

bt = Backtest(hist, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2022-10-04 00:00...
End                       2023-10-04 00:00...
Duration                    365 days 00:00:00
Exposure Time [%]                   62.396694
Equity Final [$]                 10539.779094
Equity Peak [$]                  11448.178949
Return [%]                           5.397791
Buy & Hold Return [%]               12.992556
Return (Ann.) [%]                    5.627003
Volatility (Ann.) [%]               14.491722
Sharpe Ratio                         0.388291
Sortino Ratio                         0.64465
Calmar Ratio                         0.709147
Max. Drawdown [%]                   -7.934885
Avg. Drawdown [%]                   -2.677627
Max. Drawdown Duration      146 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                        3.84227
Worst Trade [%]                     -1.613203
Avg. Trade [%]                    

## Analyzing the Results
- Equity Final [$]: This is the final equity value at the end of the backtest. It represents the total account balance after all trading activities.


- Equity Peak [$]: This is the highest equity value reached during the backtest. It indicates the peak performance of your strategy in terms of account balance.


- Return [%]: This is the total return on your investment as a percentage. It shows how much your initial capital has grown or shrunk.


- Buy & Hold Return [%]: This is the return you would have achieved if you simply bought and held the asset without any trading. It provides a benchmark for comparison.


- Return (Ann.) [%]: This is the annualized return percentage. It's the return adjusted for an annual time frame, making it easier to compare with other investments.


- Volatility (Ann.) [%]: This represents the annualized volatility of your strategy. It measures the variation in returns over time. Lower volatility is generally preferred.


- Sharpe Ratio: The Sharpe Ratio measures the risk-adjusted return of your strategy. A higher Sharpe Ratio indicates better risk-adjusted performance. It's a common metric for assessing investment strategies.


- Sortino Ratio: Similar to the Sharpe Ratio, the Sortino Ratio measures risk-adjusted return but focuses on downside risk (volatility of negative returns). A higher Sortino Ratio indicates better risk-adjusted performance, especially in strategies that aim to minimize downside risk.


- Calmar Ratio: The Calmar Ratio measures the risk-adjusted return relative to the maximum drawdown. A higher Calmar Ratio is generally better, as it indicates better returns relative to the risk of significant losses.


- Win Rate [%]: This is the percentage of profitable trades out of the total number of trades. A higher win rate is generally preferred.


- Best Trade [%]: This is the percentage return of the best individual trade in your strategy.


- Worst Trade [%]: This is the percentage return of the worst individual trade in your strategy.


- Avg. Trade [%]: This is the average percentage return of all trades in your strategy. It provides a measure of the typical trade performance.


- Max. Trade Duration: This indicates the duration (in days) of the longest individual trade.


- Avg. Trade Duration: This is the average duration (in days) of all trades.


- Profit Factor: The Profit Factor measures the ratio of gross profit to gross loss. A higher profit factor is generally desirable.


- Expectancy [%]: Expectancy represents the average amount you can expect to win (or lose) per trade.

# Interpreting the Results:

Ideal ranges for backtesting metrics can vary depending on your specific trading strategy, risk tolerance, and investment goals. However, I can provide some general guidelines for what traders often consider ideal or acceptable ranges for key backtesting metrics in a 12-month backtest. Keep in mind that these ranges are not set in stone, and what is considered ideal can differ from one trader to another.

1. **Return [%]:**
   - Ideal Range: Positive returns are typically desirable. Aiming for a return that outperforms a benchmark (e.g., Buy & Hold) is a common goal.

2. **Sharpe Ratio:**
   - Ideal Range: A Sharpe Ratio greater than 1 is often considered good, indicating a positive risk-adjusted return. Higher values are better.

3. **Sortino Ratio:**
   - Ideal Range: A Sortino Ratio greater than 1 is generally seen as positive. It focuses on minimizing downside risk, so a higher Sortino Ratio is preferred.

4. **Calmar Ratio:**
   - Ideal Range: A Calmar Ratio greater than 1 is typically desired. It assesses risk-adjusted return relative to maximum drawdown. Higher values are better.

5. **Max. Drawdown [%]:**
   - Ideal Range: A lower maximum drawdown is better. Ideally, it should be less than 20% for most traders, but this can vary based on risk tolerance.

6. **Volatility (Ann.) [%]:**
   - Ideal Range: Lower volatility is generally preferred. Volatility should be consistent with your risk tolerance and investment goals.

7. **Win Rate [%]:**
   - Ideal Range: A win rate above 50% is typically desirable. A higher win rate indicates more winning trades.

8. **Profit Factor:**
   - Ideal Range: A profit factor greater than 1 is usually considered good. It indicates that the strategy's gross profit outweighs gross losses.

9. **SQN (System Quality Number):**
   - Ideal Range: An SQN value greater than 1 is generally seen as positive. It assesses the overall quality of the trading system.

10. **Expectancy [%]:**
    - Ideal Range: A positive expectancy indicates that, on average, each trade contributes positively to the strategy's performance.

It's important to note that these ranges are general guidelines and may vary based on the specific trading strategy and risk tolerance. Some traders may be willing to accept higher drawdowns in exchange for potentially higher returns, while others prioritize lower risk and stability.

Additionally, backtesting is just one part of the evaluation process. Real-world trading involves factors like slippage, transaction costs, and market conditions that can impact performance. It's essential to consider these factors and conduct thorough analysis before implementing a trading strategy in live markets.

In [90]:
from bokeh.plotting import show

# Assuming you have created the 'bt' plot object
bt.plot()



Row(id='9946', ...)